In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index


array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
x_test_adv_0_1.shape

(911768, 56)

In [6]:
from xgboost import XGBClassifier

print('xgb')
xgb100 = XGBClassifier()
xgb100.fit(x_test_adv_0_1, y_label_0_1)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf100 = RandomForestClassifier()
rf100.fit(x_test_adv_0_1, y_label_0_1)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt100 = DecisionTreeClassifier()
dt100.fit(x_test_adv_0_1, y_label_0_1)

xgb
RF
DT


DecisionTreeClassifier()

In [7]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ExcludeCaFA/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [  2826    137   2357    818     72   1107      1    176 106183    210
     84]
(2826, 56) (2826,)
Save 0 to baseline_Def1.npy
(137, 56) (137,)
Save 1 to baseline_Def2.npy
(2357, 56) (2357,)
Save 2 to baseline_Def3.npy
(818, 56) (818,)
Save 3 to baseline_Def4.npy
(72, 56) (72,)
Save 4 to baseline_Def5.npy
(1107, 56) (1107,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(176, 56) (176,)
Save 7 to baseline_Def8.npy
(106183, 56) (106183,)
Save 8 to baseline_Def9.npy
(210, 56) (210,)
Save 9 to baseline_Def10.npy
(84, 56) (84,)
Save 10 to baseline_Def11.npy
Execution Time: 2.240692 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [132731     59   1317   1282    824 152238     97  53269     71     25]
(132731, 56) (132731,)
Save 0 to BIM_Def1.npy
(59, 56) (59,)
Save 1 to BIM_Def2.npy
(1317, 56) (1317,)
Save 2 to BIM_Def3.npy
(1282, 56) (1282,)
Save 3 to BIM_Def4.npy
(824, 56) (824,)
Save 4 to BIM_Def5.npy
(152238, 56) (1522

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [  3855    253   2247   1083    129   1544      1    321 104276    146
    116]
(3855, 56) (3855,)
Save 0 to baseline_Def1.npy
(253, 56) (253,)
Save 1 to baseline_Def2.npy
(2247, 56) (2247,)
Save 2 to baseline_Def3.npy
(1083, 56) (1083,)
Save 3 to baseline_Def4.npy
(129, 56) (129,)
Save 4 to baseline_Def5.npy
(1544, 56) (1544,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(321, 56) (321,)
Save 7 to baseline_Def8.npy
(104276, 56) (104276,)
Save 8 to baseline_Def9.npy
(146, 56) (146,)
Save 9 to baseline_Def10.npy
(116, 56) (116,)
Save 10 to baseline_Def11.npy
Execution Time: 1.938828 seconds
(341913,)
[0 1 2 3 4 5 7 8 9] [124852      5    556   1103      3 181629      1  33758      6]
(124852, 56) (124852,)
Save 0 to BIM_Def1.npy
(5, 56) (5,)
Save 1 to BIM_Def2.npy
(556, 56) (556,)
Save 2 to BIM_Def3.npy
(1103, 56) (1103,)
Save 3 to BIM_Def4.npy
(3, 56) (3,)
Save 4 to BIM_Def5.npy
(181629, 56) (181629,)
Save 5 to BIM_De

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt100.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input100/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ExcludeCaFA/UNSW_Input100/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [  7062    422   1966    489    194   1525    318 101861     57     77]
(7062, 56) (7062,)
Save 0 to baseline_Def1.npy
(422, 56) (422,)
Save 1 to baseline_Def2.npy
(1966, 56) (1966,)
Save 2 to baseline_Def3.npy
(489, 56) (489,)
Save 3 to baseline_Def4.npy
(194, 56) (194,)
Save 4 to baseline_Def5.npy
(1525, 56) (1525,)
Save 5 to baseline_Def6.npy
(318, 56) (318,)
Save 7 to baseline_Def8.npy
(101861, 56) (101861,)
Save 8 to baseline_Def9.npy
(57, 56) (57,)
Save 9 to baseline_Def10.npy
(77, 56) (77,)
Save 10 to baseline_Def11.npy
Execution Time: 0.202945 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [143327    928   4023   6287   3414 129020   2693  21665  30072    332
    152]
(143327, 56) (143327,)
Save 0 to BIM_Def1.npy
(928, 56) (928,)
Save 1 to BIM_Def2.npy
(4023, 56) (4023,)
Save 2 to BIM_Def3.npy
(6287, 56) (6287,)
Save 3 to BIM_Def4.npy
(3414, 56) (3414,)
Save 4 to BIM_Def5.npy
(129020, 56) (129020,)
Save 5 to BIM_Def6.npy
(2693, 56

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}100")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/UNSW_Input100/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input100_ExcludeCaFA.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB100,2826,137,2357,818,72,1107,1,176,106183,210,84,113971
1,XGB100,132731,59,1317,1282,824,152238,0,97,53269,71,25,341913
2,XGB100,119184,56,1282,1506,288,166606,0,783,52142,17,49,341913
3,XGB100,132731,59,1317,1282,824,152238,0,97,53269,71,25,341913
4,XGB100,74360,680,2287,377,217,242847,530,18445,1716,302,152,341913
5,XGB100,136451,278,4485,2548,935,151985,1,486,44015,489,240,341913
6,XGB100,6711,392,7065,2492,212,2996,3,524,320694,571,253,341913
7,XGB100,58813,18,691,183,17,442,1,67,281644,22,15,341913
8,XGB100,135716,343,481,798,810,148082,0,52,55561,53,17,341913
9,XGB100,112228,64,1414,1192,927,170599,0,163,55273,21,32,341913
